In [40]:
import gspread
from google.colab import auth
from google.auth.transport.requests import Request
from google.auth import default
import pandas as pd
import re
from datetime import datetime
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, Border, Side

# 1. Đăng nhập vào tài khoản Google của bạn
auth.authenticate_user()

# 2. Sử dụng thông tin xác thực mặc định của Colab
creds, _ = default()

# 3. Làm mới thông tin xác thực nếu cần
creds.refresh(Request())

# 4. Kết nối với Google Sheets
gc = gspread.authorize(creds)

# 5. Mở Google Sheet bằng URL
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1rdcNBfwBGkeMxrpGWo77I8Eyi1_3H9-57XQNOsJ-_cM/edit?gid=1968705733')

# 6. Chọn trang tính (worksheet) đầu tiên
worksheet = sheet.get_worksheet(0)  # Hoặc bạn có thể chỉ định tên trang tính

# 7. Lấy tất cả dữ liệu từ trang tính
all_data = worksheet.get_all_values()

# 8. Chuyển dữ liệu thành DataFrame
df = pd.DataFrame(all_data[1:], columns=all_data[0])

# 9. Loại bỏ khoảng trắng dư thừa trong tên cột
df.columns = df.columns.str.strip()

# 10. Chuyển đổi số liệu trong các cột cụ thể
def convert_to_int(value):
    value = re.sub(r'\D', '', value)  # Loại bỏ ký tự không phải số
    return int(value) if value.isdigit() else 0

df['Số tiết thực giảng'] = df['Số tiết thực giảng'].apply(convert_to_int)
df['Số lượng HSSV có mặt'] = df['Số lượng HSSV có mặt'].apply(convert_to_int)
df['Số lượng HSSV Vắng - Có lý do'] = df['Số lượng HSSV Vắng - Có lý do'].apply(convert_to_int)
df['Số lượng HSSV Vắng - Không có lý do'] = df['Số lượng HSSV Vắng - Không có lý do'].apply(convert_to_int)

# 11. Lọc dữ liệu theo cột "Dấu thời gian"
today = datetime.today().strftime('%d/%m/%Y')
df_today = df[df['Dấu thời gian'].str.contains(today, na=False)]

# 12. Sắp xếp các cột theo yêu cầu và thêm cột "Cán bộ chấm" đầu tiên và "Ghi chú" cuối cùng
report_columns = [
    'Cán bộ chấm',
    'Họ tên giáo viên',
    'Môn giảng theo TKB',
    'Lớp',
    'Số tiết thực giảng',
    'Số lượng HSSV có mặt',
    'Số lượng HSSV Vắng - Có lý do',
    'Số lượng HSSV Vắng - Không có lý do',
    'Ghi chú'
]
daily_report = df_today[report_columns]

# 13. Tạo workbook mới và sheet
wb = Workbook()
ws = wb.active

# 14. Thêm tiêu đề báo cáo (in hoa)
ws.merge_cells('A1:I1')
ws['A1'] = 'BÁO CÁO HÀNG NGÀY'.upper()
ws['A1'].font = Font(size=14, bold=True)
ws['A1'].alignment = Alignment(horizontal='center')

# 15. Thêm ngày tháng
ws.merge_cells('A2:I2')
ws['A2'] = f'Ngày: {today}'
ws['A2'].font = Font(size=14, bold=True)
ws['A2'].alignment = Alignment(horizontal='center')

# 16. Thêm dữ liệu từ DataFrame vào Excel và định dạng bảng
thin_border = Border(left=Side(style='thin'),
                     right=Side(style='thin'),
                     top=Side(style='thin'),
                     bottom=Side(style='thin'))

for r_idx, row in enumerate(dataframe_to_rows(daily_report, index=False, header=True), 4):
    for c_idx, value in enumerate(row, 1):
        cell = ws.cell(row=r_idx, column=c_idx, value=value)
        cell.border = thin_border
        cell.alignment = Alignment(horizontal='center')
        if r_idx == 4:  # Định dạng tiêu đề cột
            cell.font = Font(bold=True)

# 17. Thêm dòng tổng cộng
total_row = ws.max_row + 1
ws.cell(row=total_row, column=5, value="Tổng cộng").font = Font(bold=True)
ws.cell(row=total_row, column=5).alignment = Alignment(horizontal='center')
ws.cell(row=total_row, column=6, value=sum(daily_report['Số lượng HSSV có mặt'])).font = Font(bold=True)
ws.cell(row=total_row, column=6).alignment = Alignment(horizontal='center')
ws.cell(row=total_row, column=7, value=sum(daily_report['Số lượng HSSV Vắng - Có lý do'])).font = Font(bold=True)
ws.cell(row=total_row, column=7).alignment = Alignment(horizontal='center')
ws.cell(row=total_row, column=8, value=sum(daily_report['Số lượng HSSV Vắng - Không có lý do'])).font = Font(bold=True)
ws.cell(row=total_row, column=8).alignment = Alignment(horizontal='center')

# 18. Định dạng cột và dòng
for col in ws.iter_cols(min_row=4, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    max_length = 0
    column = col[0].column_letter  # Lấy chữ cái của cột
    for cell in col:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))
    adjusted_width = max_length + 2
    ws.column_dimensions[column].width = adjusted_width

# 19. Thêm ghi chú cuối bảng
last_row = ws.max_row + 2
ws.merge_cells(f'A{last_row}:I{last_row}')
ws[f'A{last_row}'] = "PHÒNG THANH TRA - CÔNG TÁC HSSV".upper()
ws[f'A{last_row}'].font = Font(bold=True)
ws[f'A{last_row}'].alignment = Alignment(horizontal='right')

# 20. Lưu file Excel vào thư mục hiện tại với ngày trong tên file
file_name = f'Báo cáo ngày_{today.replace("/", "-")}.xlsx'
wb.save(file_name)

# Lưu dữ liệu thành file CSV
csv_file_name = f'Báo cáo ngày_{today.replace("/", "-")}.csv'
daily_report.to_csv(csv_file_name, index=False)

# Thông báo về kết quả lưu file
print(f"Báo cáo hàng ngày đã được lưu thành file Excel: {file_name}")



Báo cáo hàng ngày đã được lưu thành file Excel: Báo cáo ngày_09-08-2024.xlsx
